In [42]:
#@formatter:off
%load_ext autoreload
%autoreload 2
#@formatter:on

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import pandas as pd
from mbench import (
    Parameter,
    ParameterData,
    CountryData,
    AdjacentDistricts,
    Country,
    District,
    Districts,
    OldDistrictToNewDistrict,
    demographic,
    util,
)


In [44]:
if sys.platform == 'linux':
    os.chdir('/mnt/d/Dropbox/benchmarking')
elif sys.platform =='Darwin':
    os.chdir('/Users/sepmein/Dropbox/benchmarking')

data_dir = os.path.abspath('Data')
openmalaria_xml_config_path = os.path.abspath('Data/openmalaria/12936_2017_2051_MOESM1_ESM.xml')
openmalaria_xml_config_path_pn2 = os.path.abspath("Data/openmalaria/VanDucAP2.xml")
openmalaria_xml_config_path_pn3 = os.path.abspath("Data/openmalaria/VanDucAP3.xml")

In [45]:
ahafo = District(iso='gh-af', aliases=['AHAFO'])
ashanti = District(iso='gh-ah', aliases=['ashanti'])
bono = District(iso='gh-bo', aliases=['bono'])
bono_east = District(iso='gh-be', aliases=['bono_east'])
central = District(iso='gh-cp', aliases=['central'])
eastern = District(iso='gh-ep', aliases=['eastern'])
greater_accra = District(iso='gh-aa', aliases=['greater_accra'])
north_east = District(iso='gh-ne', aliases=['north_east'])
northern = District(iso='gh-np', aliases=['northern'])
oti = District(iso='gh-ot', aliases=['oti'])
savannah = District(iso='gh-sv', aliases=['savannah'])
upper_east = District(iso='gh-ue', aliases=['upper_east'])
upper_west = District(iso='gh-uw', aliases=['upper_west'])
volta = District(iso='gh-tv', aliases=['volta'])
western = District(iso='gh-wp', aliases=['western'])
western_north = District(iso='gh-wn', aliases=['western_north'])

gha_districts = Districts(district_list=[
    ahafo, ashanti, bono, bono_east, central, eastern, greater_accra, north_east, northern, oti, savannah, upper_east, upper_west, volta, western, western_north
])
gha_districts.n

16

In [46]:
district_216_260 = pd.read_csv(data_dir + "/GHA/ADM1/216 to 260.csv")

In [47]:
gha_old_new_districts = OldDistrictToNewDistrict(
    start = district_216_260['216'],
    to= district_216_260['260'],
    districts = gha_districts,
    columns = ['old','new']
)
gha_old_new_districts.data

,old,new
0,BRONG_AHAFO,GH_AF
1,ASHANTI,GH_AH
2,BRONG_AHAFO,GH_BO
3,BRONG_AHAFO,GH_BE
4,CENTRAL,GH_CP
5,EASTERN,GH_EP
6,GREATER_ACCRA,GH_AA
7,NORTHERN,GH_NP
8,NORTHERN,GH_NE
9,VOLTA,GH_OT


In [48]:
# load adjacent
gha_adjacent_provinces_data = pd.read_excel(
    data_dir + "/gha_adm_adjacent.xlsx", sheet_name="adm1_adjacent"
)
gha_adjacent_provinces_data = demographic.reformat.adm1_name(
    df=gha_adjacent_provinces_data,
    original_column_name="a",
    new_column_name="from",
    set_index=False,
)
gha_adjacent_provinces_data = demographic.reformat.adm1_name(
    df=gha_adjacent_provinces_data,
    original_column_name="b",
    new_column_name="to",
    set_index=False,
)

gha_adjacent_provinces = AdjacentDistricts(
    start= gha_adjacent_provinces_data['from'],
    to = gha_adjacent_provinces_data['to'],
    districts = gha_districts
)

In [49]:
gha = Country(districts=gha_districts,
              adjacent_districts=gha_adjacent_provinces,
              old_districts_to_new_districts = gha_old_new_districts
              )
gha

In [50]:
gha_data = CountryData(country = gha)

In [51]:
gha_data.index

Index(['GH_AF', 'GH_AH', 'GH_BO', 'GH_BE', 'GH_CP', 'GH_EP', 'GH_AA', 'GH_NE',
       'GH_NP', 'GH_OT', 'GH_SV', 'GH_UE', 'GH_UW', 'GH_TV', 'GH_WP', 'GH_WN'],
      dtype='object')

In [52]:
gha_adm1_vector_resistance = pd.read_excel(
    data_dir
    + "/GHA/WHO_IR_data/MTM_DISCRIMINATING_CONCENTRATION_BIOASSAY_20211130.xlsx",
    sheet_name="Data",
)
gha_adm1_vector_resistance = gha_adm1_vector_resistance.groupby("ADMIN1").mean()
gha_adm1_vector_resistance = gha_adm1_vector_resistance["MORTALITY_ADJUSTED"]
gha_adm1_vector_resistance

ADMIN1
Ashanti            51.592958
Brong Ahafo        42.100000
Central            40.071053
Eastern            32.664865
Greater Accra      32.057600
Kassena Nankana    81.725000
Northern           72.059341
Upper East         53.837500
Upper West         44.305882
Volta              52.340000
Western            64.860000
Name: MORTALITY_ADJUSTED, dtype: float64

In [53]:
vector_resistance = Parameter(
    name= 'vector resistance',
    aliases= ['vr', 'vector_resistance']
)
pm_gha_vector_resistance = ParameterData(
    parameter= vector_resistance,
    data=gha_adm1_vector_resistance,
    country=gha,
    from_old_districts_system= True
)
pm_gha_vector_resistance

district
GH_AF    42.100000
GH_AH    51.592958
GH_BO    42.100000
GH_BE    42.100000
GH_CP    40.071053
GH_EP    32.664865
GH_AA    32.057600
GH_NP    72.059341
GH_NE    72.059341
GH_OT    52.340000
GH_SV    72.059341
GH_UE    53.837500
GH_UW    44.305882
GH_TV    52.340000
GH_WP    64.860000
GH_WN    64.860000
Name: MORTALITY_ADJUSTED, dtype: float64

In [54]:
incid = pd.read_csv(data_dir + "/GHA/Routine_data/District-level/incid.csv")
incid = incid[incid["year"] == 2018]
incid = demographic.reformat.adm1_name(
    df=incid, original_column_name="adm1", set_index=False
)
incid.sample(5)
gha_adm2_population = incid[["adm1", "adm2", "pop"]]
gha_adm2_population = gha_adm2_population.rename(columns={"pop": "population"})
gha_adm2_population.sample(25)

,adm1,adm2,population
731,OTI,NKWANTA NORTH,76425.0
763,ASHANTI,OFFINSO MUNICIPAL,90916.0
890,EASTERN,SUHUM MUNICIPAL,108053.0
933,AHAFO,TANO SOUTH MUNICIPAL,94928.0
637,EASTERN,KWAHU WEST MUNICIPAL,114409.0
184,EASTERN,ASENE MANSO AKROSO,98078.0
94,ASHANTI,AHAFO ANO SOUTH EAST,68370.0
193,CENTRAL,ASSIN NORTH,123641.0
1025,NORTH_EAST,YUNYOO-NASUAN,48763.0
236,ASHANTI,ATWIMA NWABIAGYA NORTH,84561.0


In [63]:
# calculate prevalence
incid = incid.groupby("adm1").sum()
prevalence = Parameter(name="Prevalence")
gha_prevalence = ParameterData(
    parameter=prevalence, data=incid["conf"] / incid["pop"], country=gha
)
gha_prevalence

adm1
GH_AF    0.316131
GH_AH    0.109697
GH_BO    0.418152
GH_BE    0.329378
GH_CP    0.221565
GH_EP    0.208044
GH_AA    0.036870
GH_NP    0.108752
GH_NE    0.108131
GH_OT    0.199682
GH_SV    0.221465
GH_UE    0.375045
GH_UW    0.317987
GH_TV    0.149964
GH_WP    0.253377
GH_WN    0.340627
Name: Prevalence, dtype: float64